## Capstone Project - Columbia DSI - DiDi - Spring 2021
#### Authors:  Danyang Han,  Anita Pinto,  Elizabeth Yum
This file is used to generate the average arrival rate for each taxi zone at a particular time zone. A day is divided into 48 30 min time zones ( delta t)  
Reference: https://github.com/skyetim/didi-vehicle-repositioning-strategy(Fall 2020 Capstone Project) 

In [3]:
import pandas as pd 
import datetime as dt 
import numpy as np
#from sqlalchemy import create_engine

In [4]:
df = pd.read_csv('../data/trip_cleaned.csv') 


In [5]:
# There are 6046 rows where exact pickup time is not recorded. 
#null_count_by_zone is a list of length 263. Each element is the number of such rows in a particular zone. 
df['pickup_datetime'] = pd.to_datetime(df['pickup_datetime'])
df['pickup_time'] = pd.to_datetime(df['pickup_datetime']).dt.time
df_time_null = df.loc[df['pickup_datetime'].dt.time == dt.time(0,0)]
null_count_by_zone = [] 
time_null = df_time_null['pickup_taxizone_id']
time_null = time_null.to_list()
for i in range(1,264):
    null_count_by_zone.append(time_null.count(i))
    
#null_count_by_zone

In [7]:
len(null_count_by_zone)

263

In [8]:
#delect all rows where exact pick_up_time is not recorded 

df = df.drop(df[df['pickup_datetime'].dt.time == dt.time(0,0)].index)

In [9]:
df['pickup_time'] = pd.to_datetime(df['pickup_datetime']).dt.time
#df.loc[:30, ['pickup_datetime', 'pickup_time']]


,pickup_datetime,pickup_time
0,2013-06-24 18:55:00,18:55:00
1,2013-06-24 19:12:00,19:12:00
2,2013-06-24 19:45:00,19:45:00
3,2013-06-24 20:17:00,20:17:00
4,2013-06-24 20:30:00,20:30:00
5,2013-06-24 20:37:00,20:37:00
6,2013-06-24 20:52:00,20:52:00
7,2013-06-24 22:07:00,22:07:00
8,2013-06-26 17:24:00,17:24:00
9,2013-06-26 17:36:00,17:36:00


In [10]:
#divide 24 hours into 48 time window, each time window is half a hour. 
time_bins = [dt.time(0,0),dt.time(0,30), dt.time(1,0), dt.time(1,30), dt.time(2,0), 
        dt.time(2,30), dt.time(3,0), dt.time(3,30), dt.time(4,0),
        dt.time(4,30), dt.time(5,0), dt.time(5,30), dt.time(6,0),
        dt.time(6,30), dt.time(7,0), dt.time(7,30), dt.time(8,0),
        dt.time(8,30), dt.time(9,0), dt.time(9,30), dt.time(10,0),
        dt.time(10,30), dt.time(11,0), dt.time(11,30), dt.time(12,0),
        dt.time(12,30), dt.time(13,0), dt.time(13,30), dt.time(14,0),
        dt.time(14,30), dt.time(15,0), dt.time(15,30), dt.time(16,0), 
        dt.time(16,30), dt.time(17,0), dt.time(17,30), dt.time(18,0),
        dt.time(18,30), dt.time(19,0), dt.time(19,30), dt.time(20,0),
        dt.time(20,30), dt.time(21,0), dt.time(21,30), dt.time(22,0),
        dt.time(22,30), dt.time(23,0), dt.time(23,30), dt.time(23,59,59)] 



In [18]:
#Split dataframe by time window. Gnenerate 48 csv, each containing all data where the pickup time is within a particular time window 
#for i in range(1, 48):
for i in range(0, 48):
    #print(i)
    time_window = df.loc[(df['pickup_time'] > time_bins[i]) & (df['pickup_time'] <= time_bins[i+1])]
    time_window.to_csv('../data/time_window_' + str(i+1) + '.csv')
    #time_window.to_csv('../data/time_window_' + str(i+1) + '.csv')


In [19]:
#test 
df_t1 = pd.read_csv('../data/time_window_1.csv')  
df_t1

t1_zone_arrival = [] 
for zone in range(1,264):
    t1_zone_arrival.append(df_t1.loc[(df_t1['pickup_taxizone_id'] == zone)].shape[0])

#t1_zone_arrival


In [17]:
#test 
df_t2 = pd.read_csv('../data/time_window_2.csv')  
df_t2

zone_t_2_arrival = [] 
for zone in range(1,264):
    zone_t_2_arrival.append(df_t2.loc[(df_t2['pickup_taxizone_id'] == zone)].shape[0])
#zone_t_2_arrival                          

In [20]:
'''
returens a list of length 263. Each element in list correspond to the
total arrival counts of time window of a particular zone over 180 days. 
parmarter "time" should be an interger in the range of 1 to 48, indicating a time window. 
For example: 1 means the time window between 0:00 and 0:30, 2 means the time window between 0:30 and 1:00, etc
zone_time_arriavl(1) returns a list of total arrival counts of each zone between 00:00 and 00:30 over the 180 days
'''
def zone_time_arrival(time):
    df_t = pd.read_csv('../data/time_window_' + str(time) + '.csv')
    zone_time_arrival = [] 
    for zone in range(1,264):
        zone_time_arrival.append(df_t.loc[(df_t['pickup_taxizone_id'] == zone)].shape[0])
    return zone_time_arrival 

In [21]:
arrival_matrix = []
for time in range(1,49):
    arrival_matrix.append(zone_time_arrival(time))
    

In [22]:
arrival_matrix = np.matrix(arrival_matrix)
np.shape(arrival_matrix)
print(arrival_matrix)
average_arrival_matrix = arrival_matrix/180
average_arrival_matrix

[[   0    1    0 ... 1168 1505 4518]
 [   0    0    1 ...  961 1179 3829]
 [   0    0    1 ...  699  904 2945]
 ...
 [   1    1    1 ... 1391 2195 5742]
 [   1    2    0 ... 1343 2034 5430]
 [   0    4    1 ... 1077 1666 4863]]


matrix([[0.00000000e+00, 5.55555556e-03, 0.00000000e+00, ...,
         6.48888889e+00, 8.36111111e+00, 2.51000000e+01],
        [0.00000000e+00, 0.00000000e+00, 5.55555556e-03, ...,
         5.33888889e+00, 6.55000000e+00, 2.12722222e+01],
        [0.00000000e+00, 0.00000000e+00, 5.55555556e-03, ...,
         3.88333333e+00, 5.02222222e+00, 1.63611111e+01],
        ...,
        [5.55555556e-03, 5.55555556e-03, 5.55555556e-03, ...,
         7.72777778e+00, 1.21944444e+01, 3.19000000e+01],
        [5.55555556e-03, 1.11111111e-02, 0.00000000e+00, ...,
         7.46111111e+00, 1.13000000e+01, 3.01666667e+01],
        [0.00000000e+00, 2.22222222e-02, 5.55555556e-03, ...,
         5.98333333e+00, 9.25555556e+00, 2.70166667e+01]])

In [23]:
pd.DataFrame(np.matrix(average_arrival_matrix)).to_csv("../data/average_arrival_matrix_48*263.csv")

In [24]:
pd.DataFrame(np.matrix(average_arrival_matrix).T).to_csv("../data/average_arrival_matrix_263*48.csv")

In [25]:
def split_by_time:
    time_window = []
    for i in range(48):
        time_window.append(df.loc[(df['pickup_time'] > time_bins[i]) & 
                                   (df['pickup_time'] <= time_bins[i+1])&  
                                   (df['pickup_taxizone_id'] == 1) ])
    time_window 
    

SyntaxError: invalid syntax (<ipython-input-25-ab09fa4254ed>, line 1)

In [ ]:
zone1_by_time_df = []
zone1_time1 = df.loc[(df['pickup_time'] <= time_bins[0]) & (df['pickup_taxizone_id'] == 4)]
zone1_by_time_df.append(zone1_time1)
for i in range(47): 
    zone1_by_time_df.append(df.loc[(df['pickup_time'] > time_bins[i]) & 
                                   (df['pickup_time'] <= time_bins[i+1])&  
                                   (df['pickup_taxizone_id'] == 4) ])

In [ ]:
zone1_by_time_df[0]

In [ ]:
zone1_by_time_df[0].loc[:, ['pickup_datetime', 'pickup_taxizone_id']]

In [ ]:
zone1_arrival_total = []
for i in range(len(zone1_by_time_df)):
    zone1_arrival_total.append(zone1_by_time_df[i].shape[0])
print(zone1_arrival_total)
zone1_arrival_rate = [zone1_arrival_total/180 for zone1_arrival_total in zone1_arrival_total ]    
#zone1_arrival_rate

In [ ]:
'''
Return a list of data frame with length 48. 
For example, zone_by_time(1)[0] returns a dataframe that contains all rows where pickup zone
is 1 and pickup_time is between 0:00 and 0:30 ''' 
def zone_by_time (zone):
    zone_by_time_df = []
    zone_time1 = df.loc[(df['pickup_time'] <= time_bins[0]) & (df['pickup_taxizone_id'] == zone)]
    zone_by_time_df.append(zone_time1)
    for i in range(47): 
       # print(i)
        zone_by_time_df.append(df.loc[(df['pickup_time'] > time_bins[i]) & 
                                   (df['pickup_time'] <= time_bins[i+1])&  
                                   (df['pickup_taxizone_id'] == zone) ])
    return zone_by_time_df
    